## Final pipeline

In [41]:
# Import main packages
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
import numpy as np
import cv2
import os

from utils import *
from classify_background import *
from segmentation import *
from classification import *

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
### TO ADAPT ###
neutral_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/downsampled/downsampled/train/neutral"
noisy_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/downsampled/downsampled/train/noisy"
hand_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/downsampled/downsampled/train/hand"

neutral_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/neutral"
noisy_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/noisy"
hand_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/hand"

all_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/all"
downsampled_all = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/downsampled/downsampled/train/all"

downsampled_test = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/downsampled/downsampled/test"
test_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/test"

model_path1 = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr_group_55/project/classifier_class3_b6_lr2e3_step7_epoch11_split0.75"
model_path2 = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr_group_55/project/resnet_b6_lr1e2_step7_epoch8_split0.75"

csv_file = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train_labels.csv"
csv_path = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/submission.csv"

Choose folder to process

In [44]:
### TO ADAPT ###
folder = all_folder
downsampled_folder = downsampled_all

Background

In [45]:
images, ids, backgrounds = load_images_and_backgrounds(folder, downsampled_folder)

Processing file L1010373.JPG
hand
Processing file L1010375.JPG
hand
Processing file L1010377.JPG
hand
Processing file L1010378.JPG
hand
Processing file L1010382.JPG
hand
Processing file L1010383.JPG
hand
Processing file L1010388.JPG
hand
Processing file L1010390.JPG
hand
Processing file L1010391.JPG
hand
Processing file L1010395.JPG
hand
Processing file L1010506.JPG
hand
Processing file L1010510.JPG
hand
Processing file L1010515.JPG
hand
Processing file L1010517.JPG
hand
Processing file L1010521.JPG
hand
Processing file L1010523.JPG
hand
Processing file L1010524.JPG
hand
Processing file L1010277.JPG
neutral
Processing file L1010279.JPG
neutral
Processing file L1010281.JPG
neutral
Processing file L1010283.JPG
neutral
Processing file L1010287.JPG
neutral
Processing file L1010288.JPG
neutral
Processing file L1010294.JPG
neutral
Processing file L1010297.JPG
neutral
Processing file L1010298.JPG
neutral
Processing file L1010300.JPG
neutral
Processing file L1010308.JPG
neutral
Processing file

Predictions

In [47]:
predictions, subclasses = predict(images, ids, backgrounds, model_path1, model_path2)

L1010373
L1010375
L1010377
L1010378
L1010382
L1010383
L1010388
L1010390
L1010391
L1010395
L1010506
L1010510
L1010515
L1010517
L1010521
L1010523
L1010524
L1010277
L1010279
L1010281
L1010283
L1010287
L1010288
L1010294
L1010297
L1010298
L1010300
L1010308
L1010310
L1010311
L1010318
L1010321
L1010323
L1010405
L1010406
L1010408
L1010410
L1010413
L1010418
L1010419
L1010421
L1010422
L1010424
L1010434
L1010436
L1010440
L1010445
L1010446
L1010450
L1010454
L1010325
L1010328
L1010331
L1010335
L1010341
L1010345
L1010349
L1010352
L1010353
L1010356
L1010361
L1010365
L1010367
L1010369
L1010370
L1010462
L1010468
L1010470
L1010475
L1010476
L1010477
L1010478
L1010479
L1010483
L1010485
L1010486
L1010487
L1010491
L1010500
L1010501
L1010503


In [48]:
predictions

{'L1010373': array([1., 2., 0.]),
 'L1010375': array([0., 3., 1.]),
 'L1010377': array([1., 3., 0.]),
 'L1010378': array([4., 1., 0.]),
 'L1010382': array([1., 0., 0.]),
 'L1010383': array([3., 2., 0.]),
 'L1010388': array([3., 1., 0.]),
 'L1010390': array([1., 3., 0.]),
 'L1010391': array([1., 2., 0.]),
 'L1010395': array([1., 3., 0.]),
 'L1010506': array([1., 1., 1.]),
 'L1010510': array([0., 2., 1.]),
 'L1010515': array([1., 1., 1.]),
 'L1010517': array([1., 1., 1.]),
 'L1010521': array([1., 1., 1.]),
 'L1010523': array([0., 2., 1.]),
 'L1010524': array([1., 1., 1.]),
 'L1010277': array([1., 4., 0.]),
 'L1010279': array([4., 0., 0.]),
 'L1010281': array([6., 2., 0.]),
 'L1010283': array([1., 5., 0.]),
 'L1010287': array([3., 5., 0.]),
 'L1010288': array([2., 3., 0.]),
 'L1010294': array([3., 2., 0.]),
 'L1010297': array([1., 3., 0.]),
 'L1010298': array([5., 0., 0.]),
 'L1010300': array([1., 4., 0.]),
 'L1010308': array([1., 8., 0.]),
 'L1010310': array([3., 4., 0.]),
 'L1010311': a

id,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD


In [49]:
subclasses

{'L1010373': array([0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'L1010375': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 0., 0., 0., 1.]),
 'L1010377': array([0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.]),
 'L1010378': array([0., 0., 3., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'L1010382': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'L1010383': array([0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.]),
 'L1010388': array([2., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'L1010390': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'L1010391': array([0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'L1010395': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.]),
 'L1010506': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]),
 'L1010510': array([0., 0., 0., 0., 0., 0., 0., 1., 0.

In [50]:
#generate_csv_file(subclasses, csv_path)

CHF predictions

In [51]:
import pandas as pd
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)
#df.drop(df.columns[8:17], axis=1, inplace=True)

selected_rows = df[df['id'].isin(subclasses.keys())]
selected_rows.shape

(81, 17)

In [56]:
# Initialize variables to store total correct predictions and total elements
total_correct = 0
total_elements = 0
all_true_labels = []
all_pred_labels = []

# Iterate over each key in subclasses
for key in subclasses.keys():
    # Extract the row values from the DataFrame
    row_values = df.loc[df['id'] == key].iloc[0, 1:17].tolist()

    # Count the number of element-wise equal elements
    equal_count = sum(1 for x, y in zip(row_values, subclasses[key]) if x == y)

    # Increment total correct and total elements
    total_correct += equal_count
    total_elements += len(subclasses[key])

    print(key)
    print(row_values)
    print(subclasses[key])

    # Collect true labels and predicted labels for F1 score computation
    all_true_labels.append(subclasses[key])
    all_pred_labels.append(row_values)

# Calculate total accuracy
total_accuracy = (total_correct / total_elements) * 100

# Compute F1 score using the custom function
f1 = compute_f1(all_pred_labels, all_true_labels)

print(f"Total accuracy: {total_accuracy:.2f}%")
print(f"F1 score: {f1:.4f}")

L1010373
[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
L1010375
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 0. 0. 1.]
L1010377
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
[0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
L1010378
[0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0. 0. 3. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
L1010382
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
L1010383
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
L1010388
[2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
L1010390
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0.]
L1010391
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
L1010395
[0, 0, 0, 1, 0, 0, 